# Day 6, cyclical number checks

- <https://adventofcode.com/2021/day/6>

Instead of decrementing the counts for each lanterfish, keep a count of the number of fishes at each life cycle stage; the example input has 1 fish each at stage 1 and 2, 2 at stage 3 and one more at stage 4. You can then either use the day number, modulo 7, as an index into these counts, or you can use a [double-ended queue](https://en.wikipedia.org/wiki/Double-ended_queue) and _rotate_ that queue for each day to move fishes between stages.

Offspring should go into a separate queue and added to the counts after two days have passed; that way they don't yet 'produce' offspring for their gestation period. You apped the 'stage 0' count onto that queue on one end, and then take the count from 2 days before and add it to the current stage.

To get the total number of fishes after such a round, sum up all the counts in the queue, and add the offspring count you'd add in the next round (these are the fishes spawned the day before but are still at stage 8).

Illustrating the example with lists instead of the Python [`deque` double-ended queue implementation](https://docs.python.org/3/library/collections.html#collections.deque), at the start we have:

```python
fishes = [0, 1, 1, 2, 1, 0, 0]
offspring = [0, 0]
```

At each step `fishes` is rotated to the left, so the value at `fishes[0]` moves to `fishes[6]` and all the other values shift along. The value at `fishes[0]` is added to the end of the `offspring` list, the `offspring[0]` value is removed and added to `fishes[0]`:

1. day 1, after rotation, append `1` to `offspring`, take the left-most `0` from `offspring` and that to `fishes[0]`:  
   `fishes`: `[1, 1, 2, 1, 0, 0, 0]`  
   `offspring`: `[0, 1]`

2. day 2, after rotation, append the old value of `fishes[0] == 1` to `offspring`, remove the left-most `0` from `offspring` and add that to `fishes[0]`:  
   `fishes`: `[1, 2, 1, 0, 0, 0, 1]`  
   `offspring`: `[1, 1]`  
   Now the sum of `fishes` is still 5, but add the `1` from `offspring[0]` and you have a population size of 6.

3. rotate, append `fishes[0] == 2` to `offspring` and pop the `1` from offspring to add to `fishes[0]`:  
   `fishes`: `[3, 1, 0, 0, 0, 1, 1]`  
   `offspring`: `[1, 2]`  
   The number of fishes is now 7.

and so on.

Together, this means we can process $n$ days in $O(n)$ time, with only $O(1)$ space required to keep track of all this.


In [1]:
from collections import deque


def simulate_fishes(starting_ages: list[int], steps: int) -> int:
    counts = [0] * 7
    for age in starting_ages:
        counts[age] += 1
    fishes = deque(counts)
    offspring = deque([0, 0])
    for _ in range(steps):
        fishes.rotate(-1)
        offspring.append(fishes[0])
        fishes[0] += offspring.popleft()
    return sum(fishes) + offspring.popleft()


test_ages = 3, 4, 3, 1, 2
assert simulate_fishes(test_ages, 18) == 26
assert simulate_fishes(test_ages, 80) == 5934

In [2]:
import aocd

start_ages = [int(n) for n in aocd.get_data(day=6, year=2021).split(",")]
print("Part 1:", simulate_fishes(start_ages, 80))

Part 1: 360268


In [3]:
assert simulate_fishes(test_ages, 256) == 26984457539

In [4]:
print("Part 1:", simulate_fishes(start_ages, 256))

Part 1: 1632146183902
